Pytorch

In [5]:
import torch
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn



image_path = './'
transform = transforms.Compose([transforms.ToTensor()])

mnist_train_dataset = torchvision.datasets.MNIST(root=image_path,
                                           train=True,
                                           transform=transform,
                                           download=True)
mnist_test_dataset = torchvision.datasets.MNIST(root=image_path,
                                           train=False,
                                           transform=transform,
                                           download=False)

batch_size = 64
torch.manual_seed(1)
train_dl = DataLoader(mnist_train_dataset, batch_size, shuffle=True)

In [6]:
#은닉층 생성  
#입력층에서 1번째 파라미터로 갈 때 갯수 : 28*28*32 +32(bias값) 
#2번째 파라미터 : 32*16 + 16
#3번째 파라미터 : 16*10 + 10 / 모두 더하면 ? 25,818 / tensorflow 파라미터와 동일 
hidden_units = [32, 16]
image_size = mnist_train_dataset[0][0].shape
input_size = image_size[0] * image_size[1] * image_size[2]

all_layers = [nn.Flatten()]
for hidden_unit in hidden_units:
    layer = nn.Linear(input_size, hidden_unit)
    all_layers.append(layer)
    all_layers.append(nn.ReLU())
    input_size = hidden_unit

#16뉴런을 10개로 
all_layers.append(nn.Linear(hidden_units[-1], 10))
model = nn.Sequential(*all_layers)

In [7]:
#손실함수 생성
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [8]:
torch.manual_seed(1)
num_epochs = 20
#epochs : 데이터셋을 모두 소진하면 1 epoch 
for epoch in range(num_epochs):
    accuracy_hist_train = 0
    #Dataload로 만든 값 train_dl / 전체 값의 파라미터를 변경
    for x_batch, y_batch in train_dl:
        pred = model(x_batch)
        #예측한 결과 값과 y값을 비교 / 사용한 함수 : CrossEntropyLoss
        loss = loss_fn(pred, y_batch)
        #역전파 
        loss.backward()
        #step하는 순간 업데이트 
        optimizer.step()
        #초기화 
        optimizer.zero_grad()
        #최종채점 
        
        is_correct = (torch.argmax(pred, dim=1) == y_batch).float()
        accuracy_hist_train += is_correct.sum()
    accuracy_hist_train /= len(train_dl.dataset)
    print(f'에포크 {epoch}  정확도 {accuracy_hist_train:.4f}')

에포크 0  정확도 0.8531
에포크 1  정확도 0.9287
에포크 2  정확도 0.9413
에포크 3  정확도 0.9506
에포크 4  정확도 0.9558
에포크 5  정확도 0.9592
에포크 6  정확도 0.9627
에포크 7  정확도 0.9650
에포크 8  정확도 0.9674
에포크 9  정확도 0.9690
에포크 10  정확도 0.9710
에포크 11  정확도 0.9729
에포크 12  정확도 0.9739
에포크 13  정확도 0.9750
에포크 14  정확도 0.9764
에포크 15  정확도 0.9777
에포크 16  정확도 0.9779
에포크 17  정확도 0.9798
에포크 18  정확도 0.9806
에포크 19  정확도 0.9813


In [10]:
#검증 데이터를 생성하여 검증 
pred = model(mnist_test_dataset.data / 255.)

In [11]:
mnist_test_dataset.data.shape

torch.Size([10000, 28, 28])

In [12]:
(mnist_test_dataset.data / 255.0).shape

torch.Size([10000, 28, 28])

In [14]:
pred = model(mnist_test_dataset.data / 1.)

is_correct = (torch.argmax(pred, dim=1) == mnist_test_dataset.targets).float()
print(f'테스트 정확도: {is_correct.mean():.4f}')

테스트 정확도: 0.9630


In [16]:
pred[0]

tensor([ -450.1693, -3686.1411,    59.7077,   122.5307, -5241.0640, -2194.6418,
        -6346.5640,  2558.0928,  -289.6331,  -286.2918],
       grad_fn=<SelectBackward0>)

In [17]:
pred[0].sum()

tensor(-15754.1729, grad_fn=<SumBackward0>)

In [18]:
torch.argmax(pred[0])

tensor(7)

In [19]:
#labelencoding 방식으로 사용됨
mnist_test_dataset.targets

tensor([7, 2, 1,  ..., 4, 5, 6])

In [24]:
#클래스로 생성하기!
#초기함수 
class mnistnet(nn.Module):
    def __init__(self):
        super(mnistnet, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 32) 
        self.fc2 = nn.Linear(32, 16)
        self.fc3 = nn.Linear(16, 10) 
        self.dropout_prob = 0.5 

    def forward(self, x):
        x = self.fc1(x.view(-1, 28*28))
        x = nn.functional.relu(x)
        x = nn.functional.dropout(x, taining=self.training, p=self.dropout_prob)
        x = self.fc2(x)
        x = nn.functional.relu(x)
         = nn.functional.dropout(x, taining=self.training, p=self.dropout_prob)
        x = self.fc3(x)
        x = nn.functional.softmax(x, dim=1)
        return x 

In [26]:
import torch
DEVICE = torch.device('cpu')
model = mnistnet().to(DEVICE)

In [27]:
model

mnistnet(
  (fc1): Linear(in_features=784, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=16, bias=True)
  (fc3): Linear(in_features=16, out_features=10, bias=True)
)

In [28]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [29]:
torch.manual_seed(1)
num_epochs = 20
#epochs : 데이터셋을 모두 소진하면 1 epoch 
for epoch in range(num_epochs):
    accuracy_hist_train = 0
    #Dataload로 만든 값 train_dl / 전체 값의 파라미터를 변경
    for x_batch, y_batch in train_dl:
        pred = model(x_batch)
        #예측한 결과 값과 y값을 비교 / 사용한 함수 : CrossEntropyLoss
        loss = loss_fn(pred, y_batch)
        #역전파 
        loss.backward()
        #step하는 순간 업데이트 
        optimizer.step()
        #초기화 
        optimizer.zero_grad()
        #최종채점 
        
        is_correct = (torch.argmax(pred, dim=1) == y_batch).float()
        accuracy_hist_train += is_correct.sum()
    accuracy_hist_train /= len(train_dl.dataset)
    print(f'에포크 {epoch}  정확도 {accuracy_hist_train:.4f}')

에포크 0  정확도 0.8253
에포크 1  정확도 0.9164
에포크 2  정확도 0.9284
에포크 3  정확도 0.9367
에포크 4  정확도 0.9424
에포크 5  정확도 0.9462
에포크 6  정확도 0.9508
에포크 7  정확도 0.9539
에포크 8  정확도 0.9567
에포크 9  정확도 0.9588
에포크 10  정확도 0.9607
에포크 11  정확도 0.9623
에포크 12  정확도 0.9638
에포크 13  정확도 0.9657
에포크 14  정확도 0.9666
에포크 15  정확도 0.9673
에포크 16  정확도 0.9684
에포크 17  정확도 0.9692
에포크 18  정확도 0.9692
에포크 19  정확도 0.9711


In [30]:
pred = model(mnist_test_dataset.data / 1.)

In [31]:
pred[0]

tensor([0., 0., 0., 0., 0., 0., 0., 1., 0., 0.], grad_fn=<SelectBackward0>)

In [32]:
is_correct = (torch.argmax(pred, dim=1) == mnist_test_dataset.targets).float()
print(f'테스트 정확도: {is_correct.mean():.4f}')

테스트 정확도: 0.9540
